In [1]:
import pandas as pd

# Load the datasets
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

# Merge datasets
data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

# Display the first few rows of the merged dataset
print(data.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region  SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36   300.68  Travis Campbell  South America  2024-04-11   
4      902.04   300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  Price_y  
0  ComfortLiving Bluetooth Speaker  Electronics   300.68  
1  ComfortLiving Bluetooth Speaker

In [2]:
# Create features for each customer
customer_features = data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'TransactionID': 'count',
    'ProductID': lambda x: len(set(x)),  # Unique products purchased
    'Region': 'first',  # Assuming region doesn't change
}).reset_index()

customer_features.columns = ['CustomerID', 'TotalSpent', 'TransactionCount', 'UniqueProducts', 'Region']

# Display the customer features
print(customer_features.head())


  CustomerID  TotalSpent  TransactionCount  UniqueProducts         Region
0      C0001     3354.52                 5               5  South America
1      C0002     1862.74                 4               4           Asia
2      C0003     2725.38                 4               4  South America
3      C0004     5354.88                 8               8  South America
4      C0005     2034.24                 3               3           Asia


In [3]:
# One-hot encoding for the Region column
customer_features = pd.get_dummies(customer_features, columns=['Region'], drop_first=True)

# Display encoded features
print(customer_features.head())


  CustomerID  TotalSpent  TransactionCount  UniqueProducts  Region_Europe  \
0      C0001     3354.52                 5               5          False   
1      C0002     1862.74                 4               4          False   
2      C0003     2725.38                 4               4          False   
3      C0004     5354.88                 8               8          False   
4      C0005     2034.24                 3               3          False   

   Region_North America  Region_South America  
0                 False                  True  
1                 False                 False  
2                 False                  True  
3                 False                  True  
4                 False                 False  


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

# Extract features for similarity calculation
features = customer_features.drop(columns=['CustomerID'])
similarity_matrix = cosine_similarity(features)

# Create a DataFrame for similarity scores
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

# Display similarity scores
print(similarity_df.head())


CustomerID  C0001     C0002     C0003  C0004  C0005     C0006     C0007  \
CustomerID                                                                
C0001         1.0  1.000000  1.000000    1.0    1.0  1.000000  1.000000   
C0002         1.0  1.000000  0.999999    1.0    1.0  0.999999  0.999999   
C0003         1.0  0.999999  1.000000    1.0    1.0  1.000000  1.000000   
C0004         1.0  1.000000  1.000000    1.0    1.0  1.000000  1.000000   
C0005         1.0  1.000000  1.000000    1.0    1.0  1.000000  1.000000   

CustomerID     C0008     C0009     C0010  ...  C0191  C0192     C0193  C0194  \
CustomerID                                ...                                  
C0001       0.999999  0.999996  0.999999  ...    1.0    1.0  1.000000    1.0   
C0002       1.000000  0.999998  1.000000  ...    1.0    1.0  0.999999    1.0   
C0003       0.999999  0.999996  0.999999  ...    1.0    1.0  1.000000    1.0   
C0004       0.999999  0.999996  0.999999  ...    1.0    1.0  1.000000    1

In [6]:
def get_top_n_similar(customers, n=3):
    recommendations = {}
    for customer in customers:
        # Get similarity scores and sort them
        similar_scores = similarity_df[customer].sort_values(ascending=False)[1:n+1]  # Exclude self-comparison
        recommendations[customer] = list(zip(similar_scores.index, similar_scores.values))
    return recommendations

# Get recommendations for customers C0001 to C0020
customer_ids = customer_features['CustomerID'].tolist()[:20]
top_lookalikes = get_top_n_similar(customer_ids)

# Convert recommendations to DataFrame for saving to CSV
# Create a list to hold the rows for the DataFrame
rows = []

for cust_id, lookalikes in top_lookalikes.items():
    for similar_cust_id, score in lookalikes:
        rows.append({'cust_id': cust_id, 'lookalike_cust_id': similar_cust_id, 'similarity_score': score})

# Create DataFrame from rows
lookalike_df = pd.DataFrame(rows)

# Save to CSV
lookalike_df.to_csv('Lookalike.csv', index=False)

# Display the resulting DataFrame
print(lookalike_df.head())


  cust_id lookalike_cust_id  similarity_score
0   C0001             C0137               1.0
1   C0001             C0152               1.0
2   C0001             C0195               1.0
3   C0002             C0139               1.0
4   C0002             C0088               1.0
